In [2]:
import pandas as pd

df_db = pd.read_csv('data/Sample_db_crisps.csv')
df_search = pd.read_csv('data/nutrition_table_crisps.csv')

In [8]:
import pandas as pd

df_search = pd.read_csv('data/Sample_db_crisps.csv')
df_db = pd.read_csv('data/nutrition_table_crisps.csv')

In [105]:
import pandas as pd

df_search = pd.read_csv('data/Sample_db.csv')
df_db = pd.read_csv('data/nutrition_table_crisps.csv')

In [113]:
import pandas as pd

df_search = pd.read_csv('data/Sample_db.csv')
df_db = pd.read_csv('data/nutrition_table.csv', encoding= 'unicode_escape' )

In [134]:
#Write some lines to encode (sentences 0 and 2 are both ideltical):
sen = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "He found a leprechaun in his walnut shell."
]
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
#Encoding:
sen_embeddings = model.encode(df_search['Product Name'].tolist())
sen_embeddings.shape

(26, 768)

In [135]:
from transformers import AutoTokenizer, AutoModel
import torch
#nitialize our model and tokenizer:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

def Bert_Embedding(df):
    # initialize dictionary: stores tokenized sentences
    token = {'input_ids': [], 'attention_mask': []}
    for index, row in df.iterrows():
        # encode each sentence, append to dictionary
        sentence = row['Product Name']
        new_token = tokenizer.encode_plus(sentence, max_length=128,
                                           truncation=True, padding='max_length',
                                           return_tensors='pt')
        token['input_ids'].append(new_token['input_ids'][0])
        token['attention_mask'].append(new_token['attention_mask'][0])
    # reformat list of tensors to single tensor
    token['input_ids'] = torch.stack(token['input_ids'])
    token['attention_mask'] = torch.stack(token['attention_mask'])

    #Process tokens through model:
    output = model(**token)

    #The dense vector representations of text are contained within the outputs 'last_hidden_state' tensor
    embeddings = output.last_hidden_state

    # To perform this operation, we first resize our attention_mask tensor:
    att_mask = token['attention_mask']
    mask = att_mask.unsqueeze(-1).expand(embeddings.size()).float()
    mask_embeddings = embeddings * mask

    #Then we sum the remained of the embeddings along axis 1:
    summed = torch.sum(mask_embeddings, 1)

    summed_mask = torch.clamp(mask.sum(1), min=1e-9)
    print(summed_mask.shape)
    mean_pooled = summed / summed_mask
    return mean_pooled.cpu().detach().numpy()

In [13]:
import numpy as np
import spacy

# Need to load the large model to get the vectors
nlp = spacy.load('en_core_web_lg')

def spacy_embedding(df):
    with nlp.disable_pipes():
        doc_vectors = np.array([nlp(text).vector for text in df['Product Name']])
    return doc_vectors

In [137]:
#row_index = ind[0][0]
#df_db['Energy'] = df_db['Energy'].astype(float)
#db_tree_row = df_db.iloc[row_index]

In [ ]:
"""
Candidate Matchers
"""

"""
KD-Tree Implemention
"""
import numpy as np
from sklearn.neighbors import KDTree
'''

db_embeded = Bert_Embedding(df_db)
print(df_search)
db_search = Bert_Embedding(df_search)
'''


df = df_db

# initialize dictionary: stores tokenized sentences
token = {'input_ids': [], 'attention_mask': []}
for index, row in df.iterrows():
    # encode each sentence, append to dictionary
    sentence = row['Product Name']
    print(sentence)
    new_token = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    token['input_ids'].append(new_token['input_ids'][0])
    token['attention_mask'].append(new_token['attention_mask'][0])
# reformat list of tensors to single tensor
token['input_ids'] = torch.stack(token['input_ids'])
token['attention_mask'] = torch.stack(token['attention_mask'])

#Process tokens through model:
output = model(**token)

#The dense vector representations of text are contained within the outputs 'last_hidden_state' tensor
embeddings = output.last_hidden_state

# To perform this operation, we first resize our attention_mask tensor:
att_mask = token['attention_mask']
mask = att_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask_embeddings = embeddings * mask

#Then we sum the remained of the embeddings along axis 1:
summed = torch.sum(mask_embeddings, 1)

summed_mask = torch.clamp(mask.sum(1), min=1e-9)
print(summed_mask.shape)
mean_pooled = summed / summed_mask
db_embeded = mean_pooled.cpu().detach().numpy()
    

df = df_search

token = {'input_ids': [], 'attention_mask': []}
for index, row in df.iterrows():
    # encode each sentence, append to dictionary
    sentence = row['Product Name']
    print(type(sentence))

    new_token = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    token['input_ids'].append(new_token['input_ids'][0])
    token['attention_mask'].append(new_token['attention_mask'][0])
# reformat list of tensors to single tensor
token['input_ids'] = torch.stack(token['input_ids'])
token['attention_mask'] = torch.stack(token['attention_mask'])

#Process tokens through model:
output = model(**token)

#The dense vector representations of text are contained within the outputs 'last_hidden_state' tensor
embeddings = output.last_hidden_state

# To perform this operation, we first resize our attention_mask tensor:
att_mask = token['attention_mask']
mask = att_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask_embeddings = embeddings * mask
#Then we sum the remained of the embeddings along axis 1:
summed = torch.sum(mask_embeddings, 1)

summed_mask = torch.clamp(mask.sum(1), min=1e-9)
print(summed_mask.shape)
mean_pooled = summed / summed_mask
db_search = mean_pooled.cpu().detach().numpy()





points = db_embeded
tree = KDTree(points, leaf_size=2, metric='euclidean')              
dist, ind = tree.query(points[:1], k=1)                
print(ind)  # indices of 3 closest neighbors
print(dist)  # distances to 3 closest neighbors

Pringles Original Sharing Crisps 200g
Pringles Sour Cream & Onion Sharing Crisps 200g
Pringles Salt & Vinegar Sharing Crisps 200g
Walkers Ready Salted Multipack Crisps 6x25g
Walkers Cheese & Onion Multipack Crisps 6x25g
Pom-Bear Original Multipack Crisps 6x13g
Walkers Classic Variety Multipack Crisps 12x25g
Kettle Chips Lightly Salted Sharing Crisps 150g
Sainsbury's Ready Salted Crisps 6x25g
Sainsbury's Gourmet Sea Salt Crisps, Taste the Difference 150g
Doritos Cool Original Sharing Tortilla Chips Crisps 180g
Sensations Thai Sweet Chilli Sharing Crisps 150g
Walkers Prawn Cocktail Multipack Crisps 6x25g
Hula Hoops Original Potato Ring Crisps 6x24g
Walkers Salt & Vinegar Multipack Crisps 6x25g
Sainsbury's Sea Salt & Cider Vinegar Crisps, Taste the Difference 150g
Sainsbury's Cheddar & Spring Onion Crisps, Taste the Difference 150g
Walkers Salt & Shake Multipack Crisps 6x24g
Hula Hoops BBQ Beef Potato Ring Crisps 6x24g
Walkers Classic Variety Multipack Crisps 22x25g
Kettle Chips Sea Salt 

In [9]:
search_row = df_db.iloc[0]

def get_score(a,b):
    return 1 - abs((a-b)/(a+b))


def Evaluator(df1, df2):
    #df1:dnanudge database; df2:sainsburies
    diff = 0
    diff = diff + get_score(df1['Energy'], df2['Energy'])
    #diff = diff + get_score(df1['Fat'], df2['Fat'])
    diff = diff + get_score(df1['Saturated fat'], df2['Saturated fat'])
    diff = diff + get_score(df1['Carbohydrate'], df2['Carbohydrate'])
    diff = diff + get_score(df1['Sugar'], df2['Sugar'])
    #diff = diff + get_score(df1['Protein'], df2['Protein'])
    diff =diff + get_score(df1['Salt'], df2['Salt'])
    
    '''
    df1['Energy'] = np.where(df1['Energy'] == df2['Energy'], 'True', 'False')
    df1['Fat'] = np.where(df1['Fat'] == df2['Fat'], 'True', 'False')
    df1['Saturated fat'] = np.where(df1['Saturated fat'] == df2['Saturated fat'], 'True', 'False')
    df1['Carbohydrate'] = np.where(df1['Carbohydrate'] == df2['Carbohydrate'], 'True', 'False')
    df1['Sugar'] = np.where(df1['Sugar'] == df2['Sugar'], 'True', 'False')
    df1['Protein'] = np.where(df1['Protein'] == df2['Protein'], 'True', 'False')
    df1['Salt'] = np.where(df1['Salt'] == df2['Salt'], 'True', 'False')
    print(df1)
    '''
    return diff

score = Evaluator(db_tree_row, search_row)
print(score)

4.055863182426773


In [126]:
array = []
def bert_embedding(df_db):
    for ind, record in df_db.iterrows():
        bert_input = tokenizer(record['Product Name'],padding='max_length', max_length = 20, 
                               truncation=True, return_tensors="pt"
                              
        #Process tokens through model:
        output = model(**bert_input)

        #The dense vector representations of text are contained within the outputs 'last_hidden_state' tensor
        embeddings = output.last_hidden_state)
        print(embeddings)
        if ind == 0:
            array = db_embeded['input_ids'][0].detach().numpy()
        else:
            array= np.vstack((array,db_embeded['input_ids'][0].detach().numpy()))
    return array

def bert_series_embedding(record):
    db_embeded = bert_input = tokenizer(record['Product Name'],padding='max_length', max_length = 20, 
                               truncation=True, return_tensors="pt")
    return db_embeded['input_ids'].detach().numpy()

SyntaxError: invalid syntax (2175741952.py, line 8)

In [ ]:
def bert_embedding2(df):
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    #Encoding:
    sen_embeddings = model.encode(df_search['Product Name'].tolist())
    return sen_embeddings

In [29]:
"""
Training and Tuning Product Matcher
"""
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.neighbors import KDTree
import time
start = time.time()

'''
BERT Tokenizer with KD-TREE
'''
total = 0
total_score = 0

db_embeded = model.encode(df_db['Product Name'].tolist())
row_sums = db_embeded.sum(axis=0)
points = db_embeded / row_sums[np.newaxis,:]
tree = KDTree(points, leaf_size=2, metric='euclidean')

start2 = time.time()
for index, query_product in df_search.iterrows():
    model = SentenceTransformer('bert-base-cased')
    #Encoding:

    query_product_embeded = model.encode([query_product['Product Name']])


    #print(query_product_embeded)
    norm = np.linalg.norm(query_product_embeded)
    dist, ind = tree.query(query_product_embeded/norm, k=1)
    row_index = ind[0][0]
    #print(row_index)
    #print(df_db)
    
    db_tree_row = df_db.iloc[row_index]
    #print(db_tree_row['Energy'] -query_product['Energy'])
    score = Evaluator(db_tree_row, query_product)
    total_score = total_score + score
    total = total+5
print(total_score/total)
print(time.time()-start2)
print(time.time()-start)

Some weights of the model checkpoint at /home/fyy/.cache/torch/sentence_transformers/bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /home/fyy/.cache/torch/sentence_transformers/bert-base-cased were not us

Some weights of the model checkpoint at /home/fyy/.cache/torch/sentence_transformers/bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /home/fyy/.cache/torch/sentence_transformers/bert-base-cased were not us

Some weights of the model checkpoint at /home/fyy/.cache/torch/sentence_transformers/bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /home/fyy/.cache/torch/sentence_transformers/bert-base-cased were not us

Some weights of the model checkpoint at /home/fyy/.cache/torch/sentence_transformers/bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /home/fyy/.cache/torch/sentence_transformers/bert-base-cased were not us

0.7035316089162497
141.58139729499817
145.90612959861755


In [16]:
"""
Training and Tuning Product Matcher
"""
import numpy as np
from sklearn.neighbors import KDTree
import time
start = time.time()

'''
Spacy with KD-TREE
'''
total = 0
total_score = 0
for index, query_product in df_search.iterrows():
    db_embeded = spacy_embedding(df_db)
    query_product_embeded = spacy_embedding(query_product)
    points = db_embeded
    tree = KDTree(points, leaf_size=2, metric='manhattan')              
    dist, ind = tree.query(query_product_embeded, k=1)
    row_index = ind[0][0]
    db_tree_row = df_db.iloc[row_index]
    #print(db_tree_row['Energy'] -query_product['Energy'])
    score = Evaluator(db_tree_row, query_product)
    total_score = total_score + score
    total = total+5
print(total_score/total)
print(time.time()-start)

0.5954573721244766
36.545873165130615


In [41]:
'''
fuzzy
'''

import numpy as np
from fuzzywuzzy import fuzz
import time
start = time.time()

'''
Spacy with KD-TREE
'''
total = 0
total_score = 0
for i, query_product in df_search.iterrows():
    current_max = 0
    index = None
    for j, db_row in df_db.iterrows():
        similarity = fuzz.ratio(query_product['Product Name'], db_row['Product Name'])
        if similarity > current_max:
            current_max = similarity
            index = j
        #print(db_tree_row['Energy'] -query_product['Energy'])
    #print(index)
    score = Evaluator(df_db.iloc[index], query_product)
    total_score = total_score + score
    total = total+5
print(total_score/total)
print(time.time()-start)


0.7577947142264981
1.2904634475708008


In [42]:
'''
cosine similarity
'''
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from fuzzywuzzy import fuzz
import time
start = time.time()

'''
cosine
'''

total = 0
total_score = 0
db_embeded = model.encode(df_db['Product Name'].tolist())
start1 = time.time()
for i, query_product in df_search.iterrows():
    current_max = 0
    index = None
    for j, db_row in df_db.iterrows():
        query_product_embeded = model.encode([query_product['Product Name']])
        #print(db_embeded[j].reshape(1,-1).shape)
        #print(query_product_embeded.shape)
        similarity = cosine_similarity(query_product_embeded, db_embeded[j].reshape(1,-1))
        if similarity > current_max:
            current_max = similarity
            index = j
        #print(db_tree_row['Energy'] -query_product['Energy'])
    #print(index)
    score = Evaluator(df_db.iloc[index], query_product)
    total_score = total_score + score
    total = total+5
print(total_score/total)
print(time.time()-start1)
print(time.time()-start)

0.7944494161511716
249.13260793685913
253.63945746421814
